In [1]:
from functions import parameters as p
from functions import crime_severity as rd
from functions import statcan_data as sd



project_parameters = p.Project_Parameters()

In [2]:
params_download_incident_violation_data = {
    'project_parameters': project_parameters,
    'data_key': 'incident_violation_detail',
}

status_ivd = sd.download_statcan_data(params_download_incident_violation_data)
zip_ivd = status_ivd.get('zipfilename', None)

In [3]:
params_read_incident_violation_data = {
    'project_parameters' : project_parameters,
    'filename': zip_ivd,
    'is_zipped': True,
}

status_rivd = rd.read_downloaded_data(params_read_incident_violation_data)

/home/rohan/.local/lib/python3.9/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
/home/rohan/.local/lib/python3.9/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [4]:
df_ivd = status_rivd.get('value',None)
display(df_ivd.head(5))
display(df_ivd.describe())
df_ivd.info()

,REF_DATE,GEO,DGUID,Violations,Statistics,UOM,UOM_ID,SCALAR_FACTOR,SCALAR_ID,VECTOR,COORDINATE,VALUE,STATUS,SYMBOL,TERMINATED,DECIMALS
0,1998,Canada,2021A000011124,"Total, all violations [0]",Actual incidents,Number,223,units,0,v44348247,1.1.1,2688540.00,NaN,NaN,NaN,0
1,1998,Canada,2021A000011124,"Total, all violations [0]","Rate per 100,000 population",Rate,257,units,0,v44396346,1.1.2,8915.12,NaN,NaN,NaN,2
2,1998,Canada,2021A000011124,"Total, all violations [0]",Percentage change in rate,Percent,239,units,0,v44391402,1.1.3,NaN,..,NaN,NaN,2
3,1998,Canada,2021A000011124,"Total, all violations [0]",Percentage contribution to the Crime Severity ...,Percent,239,units,0,v1545644596,1.1.17,100.00,NaN,NaN,NaN,2
4,1998,Canada,2021A000011124,"Total, all violations [0]",Unfounded incidents,Number,223,units,0,v1000149311,1.1.15,NaN,..,NaN,NaN,0


,REF_DATE,UOM_ID,SCALAR_ID,VALUE,SYMBOL,DECIMALS
count,7.244910e+06,7.244910e+06,7244910.0,4.602490e+06,0.0,7.244910e+06
mean,2.010844e+03,2.358238e+02,0.0,4.928159e+02,NaN,9.366576e-01
std,7.566039e+00,1.485774e+01,0.0,1.327085e+04,NaN,9.979919e-01
min,1.998000e+03,2.230000e+02,0.0,-9.991000e+01,NaN,0.000000e+00
25%,2.004000e+03,2.230000e+02,0.0,0.000000e+00,NaN,0.000000e+00
50%,2.011000e+03,2.230000e+02,0.0,0.000000e+00,NaN,0.000000e+00
75%,2.018000e+03,2.570000e+02,0.0,7.000000e+00,NaN,2.000000e+00
max,2.023000e+03,2.570000e+02,0.0,2.699428e+06,NaN,2.000000e+00


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7244910 entries, 0 to 7244909
Data columns (total 16 columns):
 #   Column         Dtype  
---  ------         -----  
 0   REF_DATE       int64  
 1   GEO            object 
 2   DGUID          object 
 3   Violations     object 
 4   Statistics     object 
 5   UOM            object 
 6   UOM_ID         int64  
 7   SCALAR_FACTOR  object 
 8   SCALAR_ID      int64  
 9   VECTOR         object 
 10  COORDINATE     object 
 11  VALUE          float64
 12  STATUS         object 
 13  SYMBOL         float64
 14  TERMINATED     object 
 15  DECIMALS       int64  
dtypes: float64(2), int64(4), object(10)
memory usage: 884.4+ MB


In [5]:
def clean_statistics_column(x):
    return (x.replace(" ", "_").lower())

statistics_col_vals_needed = ['Actual incidents','Total cleared']

columns_of_interest = [
    'REF_DATE',
    'GEO',
    'Statistics',
    'VALUE'
]

In [6]:
params_prepare_incidents_data = {
    'dataframe': df_ivd,
    'statistics_values_of_interest': statistics_col_vals_needed,
    'select_columns': columns_of_interest,
    'clean_function_dict': {
        'Statistics': clean_statistics_column,
    }
}

status_prep_ivd = rd.prepare_incidents_data(params_prepare_incidents_data)

,REF_DATE,GEO,Statistics,VALUE
0,1998,Canada,Actual incidents,2688540.00
1,1998,Canada,"Rate per 100,000 population",8915.12
2,1998,Canada,Percentage change in rate,NaN
3,1998,Canada,Percentage contribution to the Crime Severity ...,100.00
4,1998,Canada,Unfounded incidents,NaN
5,1998,Canada,Percent unfounded,NaN
6,1998,Canada,Total cleared,1073453.00
7,1998,Canada,Cleared by charge,705133.00
8,1998,Canada,Cleared otherwise,368320.00
9,1998,Canada,"Total, persons charged",631010.00


/home/rohan/Documents/workspace/python/canada_crime/functions/crime_severity.py:93: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pdf_ivd[a_key] = pdf_ivd[a_key].apply(param_clean_ivd.get(a_key))


After data preparation...


Statistics,ref_date,geo,actual_incidents,total_cleared
0,1998,Alberta [48],7721.071429,3461.390476
1,1998,British Columbia [59],14002.576190,4570.180952
2,1998,"Calgary, Alberta [48825]",2049.161905,688.276190
3,1998,Canada,71693.071429,27514.547619
4,1998,"Edmonton, Alberta [48835]",2333.490476,1036.614286
5,1998,"Greater Sudbury, Ontario [35580]",333.800000,130.957143
6,1998,"Halifax, Nova Scotia [12205]",954.647619,277.042857
7,1998,"Hamilton, Ontario [35537]",1326.752381,507.861905
8,1998,"Kitchener-Cambridge-Waterloo, Ontario [35541]",841.528571,280.638095
9,1998,"London, Ontario [35555]",1095.709524,425.252381


In [7]:
ivd_df = status_prep_ivd.get('value', {})

display(ivd_df.head(50))

Statistics,ref_date,geo,actual_incidents,total_cleared
0,1998,Alberta [48],7721.071429,3461.390476
1,1998,British Columbia [59],14002.576190,4570.180952
2,1998,"Calgary, Alberta [48825]",2049.161905,688.276190
3,1998,Canada,71693.071429,27514.547619
4,1998,"Edmonton, Alberta [48835]",2333.490476,1036.614286
5,1998,"Greater Sudbury, Ontario [35580]",333.800000,130.957143
6,1998,"Halifax, Nova Scotia [12205]",954.647619,277.042857
7,1998,"Hamilton, Ontario [35537]",1326.752381,507.861905
8,1998,"Kitchener-Cambridge-Waterloo, Ontario [35541]",841.528571,280.638095
9,1998,"London, Ontario [35555]",1095.709524,425.252381


In [8]:
import os
import os.path
import pandas as pd
from functions import utilities as u


filename = params_download_incident_violation_data.get('data_key', None)
filename += '.parquet'
output_path = os.path.join(project_parameters.data_folder, filename)

params_save_ivd_data_to_parquet = {
    'output_path': output_path,
    'input_dataframe': ivd_df,
}


u.save_pandas_data_to_parquet(params_save_ivd_data_to_parquet)

{'status': 'True',
 'value': Statistics  ref_date                                  geo  actual_incidents  \
 0               1998                         Alberta [48]       7721.071429   
 1               1998                British Columbia [59]      14002.576190   
 2               1998             Calgary, Alberta [48825]       2049.161905   
 3               1998                               Canada      71693.071429   
 4               1998            Edmonton, Alberta [48835]       2333.490476   
 ...              ...                                  ...               ...   
 1226            2023  Vancouver, British Columbia [59933]       3396.804487   
 1227            2023   Victoria, British Columbia [59935]        472.884615   
 1228            2023             Windsor, Ontario [35559]        406.445513   
 1229            2023           Winnipeg, Manitoba [46602]       1355.951923   
 1230            2023                           Yukon [60]        195.631410   
 
 Statistic

In [9]:
check_df = pd.read_parquet(f'{output_path}')

check_df.head(5)

Statistics,ref_date,geo,actual_incidents,total_cleared
0,1998,Alberta [48],7721.071429,3461.390476
1,1998,British Columbia [59],14002.576190,4570.180952
2,1998,"Calgary, Alberta [48825]",2049.161905,688.276190
3,1998,Canada,71693.071429,27514.547619
4,1998,"Edmonton, Alberta [48835]",2333.490476,1036.614286
